# Cardiac Sub-Structure Segmentation Example

## Import Modules

In [ ]:
try:
    import platipy
except:
    !pip install git+https://github.com/pyplati/platipy.git
    import platipy

from matplotlib import pyplot as plt

import SimpleITK as sitk

%matplotlib inline

from platipy.imaging.tests.data import get_lung_nifti
from platipy.imaging.projects.cardiac.run import run_hybrid_segmentation
from platipy.imaging import ImageVisualiser
from platipy.imaging.label.utils import get_com

## Download Test Data

This will download lung cancer patient CT scans, with contours of several structures.
This only has to be done once - if it is ran again don't worry, the same data will not be downloaded again!

In [ ]:
data_path = get_lung_nifti()

## Load Test Image

Read in the image we want to automatically segment


In [ ]:
test_pat_path = data_path.joinpath("LCTSC-Test-S1-201")
test_image = sitk.ReadImage(str(test_pat_path.joinpath("IMAGES/LCTSC_TEST_S1_201_0_CT_0.nii.gz")))

## Run Auto-segmentation

This will take some time, and will print updates along the way


In [ ]:
auto_structures, _ = run_hybrid_segmentation(test_image)

## Save segmentations
Optionally write the automatic segmentations to disk


In [ ]:
output_directory = test_pat_path.joinpath("substructures")
output_directory.mkdir(exist_ok=True)

for struct_name in list(auto_structures.keys()):
    sitk.WriteImage(auto_structures[struct_name], str(output_directory.joinpath(f"{struct_name}.nii.gz")))

print(f"Segmentations saved to: {output_directory}")



## Visualise Segmentations

Next, we can generate a nice figure to check what the segmentation looks like using platipy's ImageVisualiser

In [ ]:
vis = ImageVisualiser(test_image, cut=get_com(auto_structures["Heart"]))
vis.add_contour({struct: auto_structures[struct] for struct in auto_structures.keys()})
fig = vis.show()
plt.savefig(output_directory.joinpath(f"snapshot.png"))

In [ ]:
fig